아이템 기반 필터링 구현하기(작업자 : 권준호)

- 사용자 선호에 따른 장르, 감독별, 줄거리 유사도 검증 구현 

In [1]:
##########################################

# 비회원 장르별 순위 추천


In [2]:
# 1. 데이터 가져오기 
# 1-1. import 설정
import pymysql
import pandas as pd


#1-2. DB에서 데이터 가져오기
# inner join을 이용해 movie, genre 테이블 가져오기

conn = pymysql.connect(host='localhost', user='kdigital', password='mysql',
                     db='firstkino', charset='utf8') 
cursor = conn.cursor()

# sql 가져오기
sql = """
    select m.*, g.*, d.*
    from movie m
    left outer join moviegenre mg
    on m.moviecd = mg.moviecd
    left outer join genre g
    on mg.genrecd = g.genrecd
    left outer join moviedirector md
    on m.moviecd = md.moviecd
    left outer join director d
    on md.directorcd = d.directorcd
"""

# sql 실행
cursor.execute(query=sql)
rows = cursor.fetchall()

# 결과 가져오기
# for row in rows:
#     print(row)

# 데이터프레임으로 변환하기
movie_genres = pd.DataFrame(rows, columns=['movieCd', 'movieNm', 'movieNmEn', 'shortContent', 'prdtYear', 'showTm', 'openDt', 'naverScore', 'imdbScore', 'watchGradeCd', 'genreCd', 'genreNm', 'directorCd', 'directorNm', 'directorNmEn'])
movie_genres

,movieCd,movieNm,movieNmEn,shortContent,prdtYear,showTm,openDt,naverScore,imdbScore,watchGradeCd,genreCd,genreNm,directorCd,directorNm,directorNmEn
0,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,19081,D.W. 그리피스,None
1,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,42546,D.W. 그리피스,None
2,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,13,서부극(웨스턴),19081,D.W. 그리피스,None
3,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,13,서부극(웨스턴),42546,D.W. 그리피스,None
4,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,19,드라마,19081,D.W. 그리피스,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159361,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,7,코미디,40137,최기윤,None
159362,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,19,드라마,16672,최기윤,None
159363,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,19,드라마,40137,최기윤,None
159364,20219995,이멀전씨 키트,Emergency Kit,None,2021,None,0,None,None,None,4,기타,16747,전건우,None


In [3]:
# 2. 데이터 전처리하기
# 2-1. shortContent, genreNm, directNm별로 전처리

movie_genres_directors1 = movie_genres[['movieCd', 'movieNm', 'shortContent']]
movie_genres_directors2 = movie_genres[['movieCd', 'movieNm', 'genreNm']]
movie_genres_directors3 = movie_genres[['movieCd', 'movieNm', 'directorNm']]

#2-2. 결측치 ""로 대체하기
movie_genres_directors1["shortContent"] = movie_genres_directors1['shortContent'].fillna("")
movie_genres_directors2["genreNm"] = movie_genres_directors2['genreNm'].fillna("")
movie_genres_directors3["directorNm"] = movie_genres_directors3['directorNm'].fillna("")

C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [4]:
# 2-3. 유사도 검증 컬럼 전처ㅣㄹ하기
import numpy as np

# 줄거리 정리
movie_genres_directors1["shortContent"] = movie_genres_directors1.groupby(by=["movieCd", "movieNm"])["shortContent"].transform(lambda row: '$$'.join(row))
movie_genres_directors1.drop_duplicates(inplace=True)
movie_genres_directors1['shortContent'] = movie_genres_directors1['shortContent'].map(lambda sc : ','.join(np.unique(sc.split('$$'))))
movie_genres_directors1.shape

# 장르 정리
movie_genres_directors2["genreNm"] = movie_genres_directors2.groupby(by=["movieCd", "movieNm"])["genreNm"].transform(lambda row: ','.join(row))
movie_genres_directors2.drop_duplicates(inplace=True)
movie_genres_directors2['genreNm'] = movie_genres_directors2['genreNm'].map(lambda sc : ','.join(np.unique(sc.split(','))))
movie_genres_directors2.shape

# 감독정리
movie_genres_directors3["directorNm"] = movie_genres_directors3.groupby(by=["movieCd", "movieNm"])["directorNm"].transform(lambda row: ','.join(row))
movie_genres_directors3.drop_duplicates(inplace=True)
movie_genres_directors3['directorNm'] = movie_genres_directors3['directorNm'].map(lambda sc : ','.join(np.unique(sc.split(','))))
movie_genres_directors3.shape

C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

(46918, 3)

In [5]:
# 2-4. movieCd, movieNm 기준으로 통합하기
movie_genres_directors = movie_genres_directors1.merge(movie_genres_directors2, on=['movieCd', 'movieNm'], how="inner")\
                                                .merge(movie_genres_directors3, on=['movieCd', 'movieNm'], how="inner")

print( movie_genres_directors.shape )
movie_genres_directors.head()

(46918, 5)


,movieCd,movieNm,shortContent,genreNm,directorNm
0,19158001,국가의 탄생,,"드라마,서부극(웨스턴),전쟁",D.W. 그리피스
1,19188001,개의 일생,,코미디,찰리 채플린
2,19190002,여름밤의 기묘한 이야기,,공포(호러),리차드 오스왈드
3,19190003,재판장,,드라마,칼 테오도르 드레이어
4,19190007,의리적 구토,,"드라마,사극,액션",김도산


In [6]:
# 2-5. content 컬럼 생성 
# shortContent + genreNm + directorNm 통합

movie_genres_directors['content'] = movie_genres_directors["shortContent"] + movie_genres_directors["genreNm"] + movie_genres_directors["directorNm"]
movie_genres_directors.head()

# 컬럼별 ','으로 구분하여 통합시키기
colums = ["shortContent", "genreNm", "directorNm"]
movie_genres_directors['content'] = movie_genres_directors[colums].apply(lambda row: ','.join(row.values), axis=1)
movie_genres_directors

,movieCd,movieNm,shortContent,genreNm,directorNm,content
0,19158001,국가의 탄생,,"드라마,서부극(웨스턴),전쟁",D.W. 그리피스,",드라마,서부극(웨스턴),전쟁,D.W. 그리피스"
1,19188001,개의 일생,,코미디,찰리 채플린,",코미디,찰리 채플린"
2,19190002,여름밤의 기묘한 이야기,,공포(호러),리차드 오스왈드,",공포(호러),리차드 오스왈드"
3,19190003,재판장,,드라마,칼 테오도르 드레이어,",드라마,칼 테오도르 드레이어"
4,19190007,의리적 구토,,"드라마,사극,액션",김도산,",드라마,사극,액션,김도산"
...,...,...,...,...,...,...
46913,20219964,생각 끝에서 찾아오는,,드라마,이수학,",드라마,이수학"
46914,20219978,볼레로 만들기,,기타,이와,",기타,이와"
46915,20219984,어디에도 없는 시간,,드라마,장태구,",드라마,장태구"
46916,20219993,청수탕 사람들,,"드라마,코미디",최기윤,",드라마,코미디,최기윤"


In [7]:
# 2-6. 행 범위 설정하기
# 데이터가 너무 많아 램 32gb로 계산할 수 없음
# 가능 범위 안으로 자르는 작업 수행
movie_genres_directors = movie_genres_directors[0:12000]

In [8]:
# 3. 문자열 genreNm 칼럼을 Count 기반으로 피처 벡터화 변환
# CountVectorizer = 단어 벡터화 카운트 정렬
# TfidfTransformer = TF-IDF로 단어 벡터화

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

count_vect = CountVectorizer(min_df=0, ngram_range=(1,1))
director_content = count_vect.fit_transform(movie_genres_directors['content'])

tfidf_transformer = TfidfTransformer()

director_content = tfidf_transformer.fit_transform(director_content)

print(director_content.shape)

(12000, 31281)


In [9]:
# 4. 리스트 객체 문자열 변경, count 피처 벡터화하기

from sklearn.metrics.pairwise import cosine_similarity

director_content_sim = cosine_similarity(director_content, director_content)
director_content_sim[0]


array([1.        , 0.        , 0.        , ..., 0.        , 0.01659823,
       0.01909573])

In [10]:
# 5. 특정영화 정보, 유사도 뽑아내기

def find_genre_score(movie_genres_directors, sim_metrics, title_name, top_n=100):

    # 입력한 영화 인덱스
    movieNm = movie_genres_directors[movie_genres_directors['movieNm'] == title_name]
    movieNm_index = movieNm.index.values

    # 입력한 영화 유사도 데이터 프레임을 추가하기
    movie_genres_directors["similarity"] = sim_metrics[movieNm_index, :].reshape(-1,1)

    # 유사도 내림차순 정렬(상위 인덱스 100개 추출)
    temp = movie_genres_directors.sort_values(by="similarity", ascending=False)
    final_index = temp.index.values[ : top_n]

    return movie_genres_directors.iloc[final_index]

In [11]:
# 6. 특정영화 유사한 장르 영화를 평점 순위별로 추천하기

# 6-1. 해리포터와 높은 유사도를 가지는 영화 순위별 정렬하기
similar_movie = find_genre_score(movie_genres_directors, director_content_sim, '해리포터와 비밀의 방', 30)
similar_movie = similar_movie[['movieCd', 'movieNm', 'similarity', 'content']]
similar_movie





C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,movieCd,movieNm,similarity,content
10823,20020222,해리포터와 비밀의 방,1.000000,",드라마,크리스 콜럼버스"
7558,19990114,스텝맘,0.969928,",드라마,코미디,크리스 콜럼버스"
5179,19940085,미세스 다웃파이어,0.923051,",가족,드라마,크리스 콜럼버스"
5800,19950278,나인 먼쓰,0.898266,",멜로/로맨스,코미디,크리스 콜럼버스"
9823,20010291,해리포터와 마법사의 돌,0.846620,",가족,판타지,크리스 콜럼버스"
4611,19920389,나 홀로 집에 2,0.835588,",가족,스릴러,액션,코미디,크리스 콜럼버스"
4212,19910355,나홀로 집에,0.808531,",가족,스릴러,어드벤처,코미디,크리스 콜럼버스"
8463,19998967,샤논 트위드의 파워플레이,0.572583,",드라마,스릴러,크리스 보"
7799,19991054,아메리칸 무비,0.379260,",다큐멘터리,크리스 스미스"
7839,19991125,자메이카 캅,0.374219,",액션,크리스 브라운"
